## NODE sheet 1
Timo Weber, Alassane Diagne

In [15]:
import autograd.numpy as np
from autograd import grad
from scipy.optimize import minimize_scalar
import scipy.linalg as la

In [16]:
class BFGSOutput:
    def __init__(self, x, fx, converged, iter):
        self.x = x
        self.fx = fx
        self.converged = converged
        self.iter = iter

    def add_history(self,history):
        self.history = history

In [17]:
def bfgs_step(f, df, x, search_direction, method = "Armijo", armijo_param = 0.5) -> float:
    """ 
    computes BFGS step size

    Parameters:
    - f: objective function
    - df: derivative of objective function
    - x: current iterate
    - method: "full", "exact" or "Armijo"
    - armijo_param: Armijo parameter. Only needed if method = "Armijo"

    returns:
    - alpha: step size
    """
    if method == "full":
        return 1
    
    elif method == "exact":
        line_search = lambda alpha: f(x + alpha*search_direction)
        return minimize_scalar(line_search).x

    elif method == "Armijo":
        line_search = lambda alpha: f(x + alpha*search_direction)
        alpha = 1
        while line_search(alpha) > f(x) + armijo_param * alpha * np.dot(df(x), search_direction):
            alpha *= 0.5
        return alpha

In [18]:
def bfgs(f, x0, method = "Armijo", armijo_param = 0.5, max_iter = 20, tol = 1e-5, save_history = False):

    """ 
    Performs the BFGS optimization algorithm

    Parameters:
    - f: Objective function
    - x0: Initial guess for x
    - method: Method used to calculate the step size. Either "full", "exact" or "Armaijo" (default)
    - armijo_param: Only needed if method="Armajio" (default: 0.5)
    - max_iter: maximum number of iterations
    - tol: Convergence tolerance
    - save_history: If True, iteration history is returned

    returns:
    BFGS output with attributes:
    - x: Calculated minimizer
    - fx: Minimal value
    - converged: If True, method converged
    - n_iter: Number of iterations
    - (optional) history
    """

    n = x0.size

    x = x0

    df = grad(f) # compute the gradient of f
    H_inv = np.eye(n) # initialize inverse Hessian

    converged = False
    n_iter = max_iter

    if save_history:
        history = []
    
    for iter in range(max_iter):

        if save_history:
            history.append(x)
        
        if la.norm(df(x)) < tol:
            n_iter = iter
            converged = True
            break
        
        search_direction = - H_inv @ df(x)
        step_size = bfgs_step(f, df, x, search_direction, method = method, armijo_param = armijo_param)

        #perform updates
        x_old = x

        x = x + step_size * search_direction

        p = x - x_old
        q = df(x) - df(x_old)
        rho = 1 / np.dot(p,q)

        if iter == 0:
            # rescale first Hessian
            H_inv *= np.dot(p,q) / np.dot(q,q)

        H_inv = (np.eye(n) - rho * p @ q.T) @ H_inv @ (np.eye(n) - rho * p @ q.T)

    out = BFGSOutput(x, f(x), converged, n_iter)
    if save_history:
        out.add_history(history)
    return out

In [19]:
def f(x):
    return x[0]**2 + x[1]** 2

In [20]:
# extract data from measurements.dat
measurements = []
with open('measurements.dat', 'r') as file:
    for line in file:
        measurements.append((line.strip().split("  ")))

measurements = np.array(measurements).astype(float)

In [21]:
# define function
def func(x):

    def y(t,x):
        return x[2] * np.exp(-(t-x[0])**2 / x[1])
    
    val = 0
    ts = measurements[:,0]
    ys = measurements[:,1]
    val = np.array([(y(ts[i],x) - ys[i])**2 for i in range(len(ts))], dtype='d')

    return np.sum(val)

In [22]:
result = bfgs(func, np.ones(3), tol=1e-3, method="exact")

/var/folders/kf/2r_v79wn3v1g_w1ngzsh6vlm0000gn/T/ipykernel_44045/502937074.py:10: RuntimeWarning: overflow encountered in scalar power
  val = np.array([(y(ts[i],x) - ys[i])**2 for i in range(len(ts))], dtype='d')
/Users/alassanediagne/miniconda3/envs/MachineLearning/lib/python3.12/site-packages/autograd/tracer.py:48: RuntimeWarning: overflow encountered in exp
  return f_raw(*args, **kwargs)
/Users/alassanediagne/miniconda3/envs/MachineLearning/lib/python3.12/site-packages/scipy/optimize/_optimize.py:2473: RuntimeWarning: invalid value encountered in scalar multiply
  tmp2 = (x - v) * (fx - fw)
/var/folders/kf/2r_v79wn3v1g_w1ngzsh6vlm0000gn/T/ipykernel_44045/2276039243.py:57: RuntimeWarning: divide by zero encountered in scalar divide
  rho = 1 / np.dot(p,q)
/var/folders/kf/2r_v79wn3v1g_w1ngzsh6vlm0000gn/T/ipykernel_44045/2276039243.py:63: RuntimeWarning: invalid value encountered in multiply
  H_inv = (np.eye(n) - rho * p @ q.T) @ H_inv @ (np.eye(n) - rho * p @ q.T)


In [24]:
result.iter

20